# Práctica módulo Despliegue de Algoritmos KeepCoding
Felipe Gallegos


Importamos librerías necesarias

In [8]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score

from sklearn.datasets import load_wine
from scipy.stats import uniform, randint

In [9]:
# Se usará el dataset iris para esta práctica
data = load_wine()
X = data.data
y = data.target


In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
n_estimators = 400
learning_rate = 0.1
max_depth = 8
subsample = 0.8



pipeline = Pipeline([
    ('scaler', StandardScaler()),  
    ('classifier', GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    ))  
])

In [108]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 GradientBoostingClassifier(max_depth=8, n_estimators=400,
                                            subsample=0.8))])

In [109]:
y_pred = pipeline.predict(X_test)

In [ ]:
# Puntajes
accuracy_train = accuracy_score(y_train, pipeline.predict(X_train))
accuracy_test = accuracy_score(y_test, y_pred)
print(f"Training Accuracy: {accuracy_train:.4f}")
print(f"Test Accuracy: {accuracy_test:.4f}")


Training Accuracy: 1.0000
Test Accuracy: 1.0000


In [ ]:
# Evalua
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 1.0000


In [112]:
accuracy_train = pipeline.score(X_train,y_train)

In [113]:
accuracy_test = pipeline.score(X_test,y_test)

In [114]:
accuracy_train, accuracy_test

(1.0, 1.0)

# Comienza MLFlow

In [103]:
input_example = X_test[0].reshape(1, -1)

mlflow.set_experiment("Wine_Classifier")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run(run_name=f"estimators:{n_estimators}, lr:{learning_rate}, max_depth:{max_depth}"):
    # Log hyperparameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("subsample", subsample)
    mlflow.log_param("test_size", 0.3) 
    
    # Log metrics
    mlflow.log_metric("accuracy train", accuracy_train)
    mlflow.log_metric("accuracy test", accuracy_test)
    
     # Log feature importances (use the trained model from the pipeline)
    feature_importance = pipeline.named_steps['classifier'].feature_importances_
    for idx, importance in enumerate(feature_importance):
        mlflow.log_metric(f"feature_{idx}_importance", importance)

    # Log the trained model
    signature = mlflow.models.signature.infer_signature(input_example, pipeline.predict(input_example))
    mlflow.sklearn.log_model(pipeline, "wine_gb_model", signature=signature, input_example=input_example)
    

🏃 View run estimators:250, lr:0.2, max_depth:6 at: http://127.0.0.1:5000/#/experiments/295359924997740381/runs/90f0f3a4c36f457db5c7dd0dee7a270d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/295359924997740381


In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine
from itertools import product


mlflow.set_tracking_uri('http://127.0.0.1:5000')  


data = load_wine()
X = data.data
y = data.target

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
input_example = np.array([X_train[0]])

param_grid = {
    'n_estimators': [80, 100, 130],
    'min_samples_leaf': [2, 5],
    'max_depth': [10, 15]
}

# combinación hyperparams
param_combinations = product(*param_grid.values())
param_names = list(param_grid.keys())

experiment_name = 'Iteracion_hyp2'
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

mlflow.set_experiment(experiment_id)

for combination in param_combinations:

    param_dict = dict(zip(param_names, combination))

    with mlflow.start_run() as run:
       
        clf = GradientBoostingClassifier(
            n_estimators=param_dict['n_estimators'],
            min_samples_leaf=param_dict['min_samples_leaf'],
            max_depth=param_dict['max_depth'],
            random_state=123
        )

        preprocessor = Pipeline(steps=[('scaler', StandardScaler())])
        model = Pipeline(steps=[('preprocessor', preprocessor), ('GradientBoostingClassifier', clf)])

        
        model.fit(X_train, y_train)

        
        accuracy_train = accuracy_score(y_train, model.predict(X_train))
        accuracy_test = accuracy_score(y_test, model.predict(X_test))

        
        mlflow.log_metric('accuracy_train', accuracy_train)
        mlflow.log_metric('accuracy_test', accuracy_test)

        for param_name, param_value in param_dict.items():
            mlflow.log_param(param_name, param_value)

        # Log 
        mlflow.sklearn.log_model(model, 'clf-model', input_example=input_example)

        
        print(f"Completed run with n_estimators={param_dict['n_estimators']}, min_samples_leaf={param_dict['min_samples_leaf']}, max_depth={param_dict['max_depth']}")


Completed run with n_estimators=80, min_samples_leaf=2, max_depth=10
🏃 View run dashing-doe-443 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/209e01066c3e490e855a9b7d82b81e19
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Completed run with n_estimators=80, min_samples_leaf=2, max_depth=15
🏃 View run popular-shad-772 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/16569092b89a422cbf1d68f8b23c995a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Completed run with n_estimators=80, min_samples_leaf=5, max_depth=10
🏃 View run smiling-moth-452 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/6fc1e36c4c5b492a95de6943d1a2306a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Completed run with n_estimators=80, min_samples_leaf=5, max_depth=15
🏃 View run mercurial-shrimp-706 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/d4286f5bff7f4b60b3da5f728d5c763c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Completed run with n_estimators=100, min_samples_leaf=2, max_depth=10
🏃 View run fearless-lark-191 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/0af06a2bd2404d699aa994bb83d0cdac
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


...

Completed run with n_estimators=100, min_samples_leaf=5, max_depth=15
🏃 View run victorious-sow-935 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/580dbb34e0284c7dbbd44cb9b5002ea1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Completed run with n_estimators=130, min_samples_leaf=2, max_depth=10
🏃 View run bright-mule-833 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/d7a8439e98864eb689f6cf5ea2626c54
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Completed run with n_estimators=130, min_samples_leaf=2, max_depth=15
🏃 View run nervous-croc-743 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/5fc0c668fd694091829cb7ceb69b6a67
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Completed run with n_estimators=130, min_samples_leaf=5, max_depth=10
🏃 View run legendary-yak-826 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/720463fdc4484af297df8b36e6c243fd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Completed run with n_estimators=130, min_samples_leaf=5, max_depth=15
🏃 View run classy-chimp-390 at: http://127.0.0.1:5000/#/experiments/663785587514068632/runs/72b1ce7a017e46a49bf5f62b340d347b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/663785587514068632


Pantallazos de respaldo

<img src="img/0_cli_ss.png" alt="CLI Screenshot" width="900">

<img src="img/0_mlfl.png" alt="MLFlow Screenshot" width="900">

<img src="img/4post.png" alt="Post Method" width="900">

<img src="img/4post1.png" alt="Post Method" width="900">
<img src="img/4post2.png" alt="Post Method" width="900">
<img src="img/4post3.png" alt="Post Method" width="900">
<img src="img/4post4.png" alt="Post Method" width="900">
<img src="img/get.png" alt="Get Method" width="900">
<img src="img/mlflow1.png" alt="MLFlow Screenshot" width="900">
<img src="img/mlflow2.png" alt="MLFlow Screenshot" width="900">
<img src="img/mlflow3.png" alt="MLFlow Screenshot" width="900">
<img src="img/mlflow4.png" alt="MLFlow Screenshot" width="900">
<img src="img/mlflow5.png" alt="MLFlow Screenshot" width="900">
<img src="img/mlflowss.png" alt="MLFlow Screenshot" width="900">